In [1]:
qpath = r'C:/Users/rgoix/DEV/quetzal/' # path to quetzal here
data = r'inputs/'
import sys
sys.path.append(qpath + r'quetzal/')

# import class
from quetzal.io.gtfs_reader import importer

### Read GTFS

In [2]:
feed = importer.GtfsImporter(path=data + r'accra_trotro', dist_units='m')
feed.clean()
feed.describe()

,indicator,value
0,agencies,[AMA Department of Transport]
1,running_services,[0]
2,timezone,Africa/Accra
3,start_date,20190304
4,end_date,20200131
5,num_routes,277
6,num_trips,554
7,num_stops,4171
8,num_shapes,554
9,num_frequencies,554


In [3]:
feed.map_stops(feed.stops.stop_id)

To simplify a GTFS or for modeling purposes, we often want to group trips that are not geographically identical, but close to each other.  
The default method to build patterns only group geographically identical trips. It however possible to clusterize the station in order to broaden the definition of a pattern.

####  Stop clustering

This method allows to clusterize the stops based on a distance threshold (and not a number of cluster threshold).
It is however not efficient for large feeds.  

Tested and working up to 5000 stops until now.

In [4]:
feed.build_stop_clusters(distance_threshold=300) # It is also possible to use the parent stations

This adds a cluster_id column to the stops

In [5]:
feed.stops.head()

,stop_name,stop_id,location_type,parent_station,cluster_id,stop_lon,stop_lat
0,Tse Addo,5010786347,0,SA5010786347,1081,-0.138535,5.587618
1,Bostal,5030747563,0,SA5030747563,358,-0.188797,5.606415
2,Second Juction,5010786348,0,SA5010786348,60,-0.141622,5.585648
3,Junction,5010786349,0,SA5010786349,60,-0.142669,5.584328
4,Fadama,SA4990070170,1,NaN,732,-0.240793,5.601748


#### Patterns

Patterns definition can be based on this new column

In [6]:
feed.build_patterns(on='cluster_id')

#### Directions

It can also be used to derive the directions of the trips if they are not known

In [7]:
feed.build_directions(on='cluster_id', all_trips=True)  # With all_trips=False, we only compute undefined directions

In [8]:
feed.trips.head()

,route_id,trip_id,service_id,shape_id,trip_headsign,pattern_id,direction_id
0,7589525,344,0,7589522,Nima Overhead Maamobi,7589525_0,1
1,7437148,0,0,7437119,Amasaman,7437148_0,1
2,7435747,346,0,7435456,Ashaiman Main Station,7435747_0,0
3,7435747,347,0,7435650,Abeka Lapaz,7435747_1,1
4,7476946,340,0,7476945,Nima,7476946_0,0
